<a href="https://colab.research.google.com/github/varsha-282/CIP/blob/main/cipdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

file_path = "Gestational DM.csv"
df = pd.read_csv(file_path)

label_encoded = df.copy()
label_encoded['Preterm Pregnancy'] = label_encoded['Preterm Pregnancy'].astype('category').cat.codes

print("Label Encoded 'Preterm Pregnancy' column:")
print(label_encoded['Preterm Pregnancy'])


Label Encoded 'Preterm Pregnancy' column:
0      1
1      0
2      1
3      0
4      1
      ..
194    1
195    0
196    1
197    1
198    1
Name: Preterm Pregnancy, Length: 199, dtype: int8


In [2]:
def mean_imputation(df):
    column_means = df.mean()
    df_filled = df.fillna(column_means)
    return df_filled

df_imputed = mean_imputation(label_encoded)
changed_values = pd.concat([label_encoded[label_encoded != df_imputed].stack(), df_imputed[df_imputed != label_encoded].stack()], axis=1)
changed_values.columns = ['Original', 'Imputed']

print("Changed values before and after mean imputation:")
print(changed_values)

Changed values before and after mean imputation:
         Original    Imputed
10  BMI       NaN  21.991837
22  BMI       NaN  21.991837
130 BMI       NaN  21.991837


In [3]:
preprocessed_file_path = "preprocessed_data.csv"
df_imputed.to_csv(preprocessed_file_path, index=False)
print(f"Preprocessed data saved to '{preprocessed_file_path}'")

Preprocessed data saved to 'preprocessed_data.csv'


In [5]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.optimizers import Adam
latent_dim = 100
def build_generator():
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(2,))
    x = concatenate([noise, label])
    x = Dense(128, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(12, activation='linear')(x)
    generator = Model([noise, label], x)
    return generator

def build_discriminator():
    data = Input(shape=(12,))
    x = Dense(256, activation='relu')(data)
    x = Dense(128, activation='relu')(x)
    validity = Dense(1, activation='sigmoid')(x)
    discriminator = Model(data, validity)
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    return discriminator

def build_combined(generator, discriminator):
    discriminator.trainable = False
    noise = Input(shape=(latent_dim,))
    label = Input(shape=(2,))
    synthetic_data = generator([noise, label])
    validity = discriminator(synthetic_data)
    combined = Model([noise, label], validity)
    combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return combined

def train_discriminator(discriminator, real_data, synthetic_data):
    real_labels = np.ones((real_data.shape[0], 1))
    fake_labels = np.zeros((synthetic_data.shape[0], 1))
    discriminator.train_on_batch(real_data, real_labels)
    discriminator.train_on_batch(synthetic_data, fake_labels)

def train_generator(combined, noise, labels):
    valid_labels = np.ones((noise.shape[0], 1))
    combined.train_on_batch([noise, labels], valid_labels)

def generate_synthetic_data(generator, num_samples):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    labels = np.random.randint(0, 2, (num_samples, 2))
    synthetic_data = generator.predict([noise, labels])
    return synthetic_data

preprocessed_data = pd.read_csv('preprocessed_data.csv')

generator = build_generator()
discriminator = build_discriminator()
combined = build_combined(generator, discriminator)

num_epochs = 10000
batch_size = 128
for epoch in range(num_epochs):
    idx = np.random.randint(0, preprocessed_data.shape[0], batch_size)
    real_batch = preprocessed_data.iloc[idx]
    synthetic_batch = generate_synthetic_data(generator, batch_size)
    train_discriminator(discriminator, real_batch, synthetic_batch)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    labels = np.random.randint(0, 2, (batch_size, 2))
    train_generator(combined, noise, labels)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{num_epochs}")

num_samples = 5000
synthetic_data = generate_synthetic_data(generator, num_samples)
synthetic_df = pd.DataFrame(synthetic_data, columns=preprocessed_data.columns)
synthetic_df.to_csv('synthetic_data.csv', index=False)


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 3ms/step
Epoch 5100/10000
4/4 [==============================] - 0s 4ms/step
Epoch 5200/10000
4/4 [==============================] - 0s 3ms/step
Epoch 5300/10000
4/4 [==============================] - 0s 3ms/step
Epoch 5400/10000
4/4 [==============================] - 0s 3ms/step
Epoch 5500/10000
4/4 [==============================] - 0s 4ms/step
Epoch 5600/10000
4/4 [==============================] - 0s 3ms/step
Epoch 5700/10000
4/4 [==============================] - 0s 4ms/step
Epoch 5800/10000
4/4 [==============================] - 0s 8ms/step
Epoch 5900/10000
4/4 [==============================] - 0s 4ms/step
Epoch 6000/10000
4/4 [==============================] - 0s 3ms/step
Epoch 6100/10000
4/4 [==============================] - 0s 3ms/step
Epoch 6200/10000
4/4 [==============================] - 0s 3ms/step
Epoch 6300/10000
4/4 [==============================] - 0s 3ms/step
Epoch 6400/100

In [6]:
synthetic_data = pd.read_csv('synthetic_data.csv')
columns_to_round = ['Age', 'No of Pregnancy', 'Preterm Pregnancy', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']
synthetic_data.columns = synthetic_data.columns.str.strip()
synthetic_data[columns_to_round] = synthetic_data[columns_to_round].round()
synthetic_data['Preterm Pregnancy'] = synthetic_data['Preterm Pregnancy'].apply(lambda x: 1 if x > 0.5 else 0)
synthetic_data.to_csv('synthetic_data_rounded.csv', index=False)


In [7]:
preprocessed_data = pd.read_csv('synthetic_data_rounded.csv')
preterm_pregnancy_counts = preprocessed_data['Preterm Pregnancy'].value_counts()

print("Counts of 1's and 0's in 'Preterm Pregnancy' column:")
print(preterm_pregnancy_counts)


Counts of 1's and 0's in 'Preterm Pregnancy' column:
Preterm Pregnancy
1    3362
0    1638
Name: count, dtype: int64


In [8]:
import pandas as pd
from imblearn.over_sampling import ADASYN
data = pd.read_csv('synthetic_data_rounded.csv')

feature_columns = ['Age', 'No of Pregnancy', 'BMI', 'Systolic Blood Pressure', 'Diastolic Blood Pressure',
                   'Bloodc Sugar Fasting', 'Blood Sugar Post Prandial', 'Oral Gucose tolerance test',
                   'C-Reactive Protein', 'Interleukin 6-IL 6', 'Birth weight']
target_column = 'Preterm Pregnancy'  # Replace 'Target' with the name of your target column
X = data[feature_columns]
y = data[target_column]
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y)
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=feature_columns),
                           pd.DataFrame(y_resampled, columns=[target_column])], axis=1)
balanced_data.to_csv('balanced_dataset.csv', index=False)



In [10]:
preprocessed_data = pd.read_csv('balanced_dataset.csv')
preterm_pregnancy_counts = preprocessed_data['Preterm Pregnancy'].value_counts()

print("Counts of 1's and 0's in 'Preterm Pregnancy' column:")
print(preterm_pregnancy_counts)


Counts of 1's and 0's in 'Preterm Pregnancy' column:
Preterm Pregnancy
1    3362
0    3066
Name: count, dtype: int64


In [12]:
pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 914.7 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [13]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.1 MB/s eta 0:00:00


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load your CSV file
data = pd.read_csv('balanced_dataset.csv')

# Split into features and target
X = data.drop(columns=['Preterm Pregnancy'])
y = data['Preterm Pregnancy']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define TabNet objective function
def objective(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64),
        "n_a": trial.suggest_int("n_a", 8, 64),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 0.1, 2.0),
        "momentum": trial.suggest_float("momentum", 0.01, 0.5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.1),
        "optimizer_params": {
            "lr": trial.suggest_float("lr", 1e-4, 1e-1, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True),
        },
    }

    model = TabNetClassifier(**params)
    model.fit(X_train.values, y_train.values, eval_set=[(X_test.values, y_test.values)], patience=10, max_epochs=100,
              batch_size=128, virtual_batch_size=32, num_workers=0, drop_last=False)

    # Evaluate model on test set
    y_pred_proba = model.predict_proba(X_test.values)
    auc = roc_auc_score(y_test, y_pred_proba[:, 1])
    return auc

# Define Optuna study with Hyperband pruner
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=50)

# Get best parameters
best_params = study.best_params
print("Best parameters:", best_params)

# Get best AUC
best_auc = study.best_value
print("Best AUC:", best_auc)


[I 2024-04-22 16:39:29,726] A new study created in memory with name: no-name-2e3a4f81-2f0f-4fb0-8c25-39bb605c7e77
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.96531 | val_0_auc: 0.48406 |  0:00:11s
epoch 1  | loss: 0.62519 | val_0_auc: 0.694   |  0:00:19s
epoch 2  | loss: 0.56922 | val_0_auc: 0.80162 |  0:00:23s
epoch 3  | loss: 0.52566 | val_0_auc: 0.83936 |  0:00:28s
epoch 4  | loss: 0.51248 | val_0_auc: 0.8494  |  0:00:32s
epoch 5  | loss: 0.51629 | val_0_auc: 0.84572 |  0:00:36s
epoch 6  | loss: 0.50368 | val_0_auc: 0.86905 |  0:00:40s
epoch 7  | loss: 0.49798 | val_0_auc: 0.87611 |  0:00:44s
epoch 8  | loss: 0.49328 | val_0_auc: 0.86937 |  0:00:49s
epoch 9  | loss: 0.49908 | val_0_auc: 0.86643 |  0:00:53s
epoch 10 | loss: 0.49222 | val_0_auc: 0.859   |  0:00:57s
epoch 11 | loss: 0.48889 | val_0_auc: 0.8773  |  0:01:02s
epoch 12 | loss: 0.49055 | val_0_auc: 0.87041 |  0:01:05s
epoch 13 | loss: 0.4877  | val_0_auc: 0.87401 |  0:01:09s
epoch 14 | loss: 0.48842 | val_0_auc: 0.85882 |  0:01:14s
epoch 15 | loss: 0.48235 | val_0_auc: 0.87379 |  0:01:18s
epoch 16 | loss: 0.47637 | val_0_auc: 0.88541 |  0:01:22s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-04-22 16:42:57,843] Trial 0 finished with value: 0.8973796999805181 and parameters: {'n_d': 19, 'n_a': 42, 'n_steps': 10, 'gamma': 0.5730781015968962, 'momentum': 0.2524537390129846, 'lambda_sparse': 0.02274035702041016, 'lr': 0.00749345912740428, 'weight_decay': 0.0008117553953782184}. Best is trial 0 with value: 0.8973796999805181.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.3317  | val_0_auc: 0.57395 |  0:00:02s
epoch 1  | loss: 0.96477 | val_0_auc: 0.55488 |  0:00:05s
epoch 2  | loss: 0.85537 | val_0_auc: 0.57519 |  0:00:09s
epoch 3  | loss: 0.78028 | val_0_auc: 0.60776 |  0:00:12s
epoch 4  | loss: 0.72323 | val_0_auc: 0.70233 |  0:00:15s
epoch 5  | loss: 0.70332 | val_0_auc: 0.71224 |  0:00:18s
epoch 6  | loss: 0.79101 | val_0_auc: 0.69819 |  0:00:22s
epoch 7  | loss: 0.71831 | val_0_auc: 0.71687 |  0:00:25s
epoch 8  | loss: 0.69878 | val_0_auc: 0.70337 |  0:00:28s
epoch 9  | loss: 0.67228 | val_0_auc: 0.70631 |  0:00:31s
epoch 10 | loss: 0.66288 | val_0_auc: 0.73765 |  0:00:35s
epoch 11 | loss: 0.64983 | val_0_auc: 0.72372 |  0:00:38s
epoch 12 | loss: 0.67164 | val_0_auc: 0.70828 |  0:00:41s
epoch 13 | loss: 0.67502 | val_0_auc: 0.71625 |  0:00:44s
epoch 14 | loss: 0.64682 | val_0_auc: 0.72219 |  0:00:48s
epoch 15 | loss: 0.6513  | val_0_auc: 0.7247  |  0:00:51s
epoch 16 | loss: 0.6424  | val_0_auc: 0.691   |  0:00:54s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-04-22 16:44:06,274] Trial 1 finished with value: 0.737646113383986 and parameters: {'n_d': 22, 'n_a': 26, 'n_steps': 8, 'gamma': 1.42641629409613, 'momentum': 0.32845745371069685, 'lambda_sparse': 0.050128867609236794, 'lr': 0.002529911345403689, 'weight_decay': 6.300357145521684e-06}. Best is trial 0 with value: 0.8973796999805181.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 1.03402 | val_0_auc: 0.48479 |  0:00:03s
epoch 1  | loss: 0.78023 | val_0_auc: 0.62005 |  0:00:06s
epoch 2  | loss: 0.71932 | val_0_auc: 0.64571 |  0:00:11s
epoch 3  | loss: 0.67486 | val_0_auc: 0.70028 |  0:00:14s
epoch 4  | loss: 0.63387 | val_0_auc: 0.77646 |  0:00:18s
epoch 5  | loss: 0.62588 | val_0_auc: 0.79359 |  0:00:22s
epoch 6  | loss: 0.58509 | val_0_auc: 0.79823 |  0:00:26s
epoch 7  | loss: 0.55784 | val_0_auc: 0.83499 |  0:00:29s
epoch 8  | loss: 0.53352 | val_0_auc: 0.83214 |  0:00:33s
epoch 9  | loss: 0.51983 | val_0_auc: 0.84836 |  0:00:37s
epoch 10 | loss: 0.50132 | val_0_auc: 0.84965 |  0:00:40s
epoch 11 | loss: 0.50046 | val_0_auc: 0.85462 |  0:00:44s
epoch 12 | loss: 0.47482 | val_0_auc: 0.85774 |  0:00:48s
epoch 13 | loss: 0.48646 | val_0_auc: 0.85755 |  0:00:51s
epoch 14 | loss: 0.47145 | val_0_auc: 0.85967 |  0:00:55s
epoch 15 | loss: 0.46972 | val_0_auc: 0.8549  |  0:00:59s
epoch 16 | loss: 0.45539 | val_0_auc: 0.85526 |  0:01:02s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-04-22 19:42:34,997] Trial 48 finished with value: 0.9044515877654392 and parameters: {'n_d': 53, 'n_a': 26, 'n_steps': 8, 'gamma': 0.7187027380435175, 'momentum': 0.07950814487134208, 'lambda_sparse': 0.006153687644072748, 'lr': 0.002872890259203326, 'weight_decay': 5.033965785015718e-05}. Best is trial 41 with value: 0.920156341320865.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.90609 | val_0_auc: 0.60793 |  0:00:02s
epoch 1  | loss: 0.69703 | val_0_auc: 0.64457 |  0:00:04s
epoch 2  | loss: 0.63247 | val_0_auc: 0.65456 |  0:00:06s
epoch 3  | loss: 0.59235 | val_0_auc: 0.74979 |  0:00:08s
epoch 4  | loss: 0.58991 | val_0_auc: 0.75975 |  0:00:11s
epoch 5  | loss: 0.58339 | val_0_auc: 0.76079 |  0:00:13s
epoch 6  | loss: 0.56411 | val_0_auc: 0.77171 |  0:00:16s
epoch 7  | loss: 0.55271 | val_0_auc: 0.79176 |  0:00:18s
epoch 8  | loss: 0.55491 | val_0_auc: 0.8061  |  0:00:20s
epoch 9  | loss: 0.54086 | val_0_auc: 0.80074 |  0:00:23s
epoch 10 | loss: 0.54419 | val_0_auc: 0.80397 |  0:00:25s
epoch 11 | loss: 0.53201 | val_0_auc: 0.8035  |  0:00:28s
epoch 12 | loss: 0.54342 | val_0_auc: 0.81552 |  0:00:30s
epoch 13 | loss: 0.53321 | val_0_auc: 0.7842  |  0:00:32s
epoch 14 | loss: 0.52331 | val_0_auc: 0.81424 |  0:00:35s
epoch 15 | loss: 0.52227 | val_0_auc: 0.81881 |  0:00:37s
epoch 16 | loss: 0.51883 | val_0_auc: 0.81528 |  0:00:39s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-04-22 19:46:49,702] Trial 49 finished with value: 0.8928185271770894 and parameters: {'n_d': 62, 'n_a': 47, 'n_steps': 4, 'gamma': 1.0016219774906734, 'momentum': 0.11594364994876019, 'lambda_sparse': 0.015067918924733945, 'lr': 0.009871861187709519, 'weight_decay': 2.8464473722012698e-05}. Best is trial 41 with value: 0.920156341320865.


Best parameters: {'n_d': 63, 'n_a': 27, 'n_steps': 8, 'gamma': 0.4802970683329498, 'momentum': 0.09655998085051334, 'lambda_sparse': 0.00962206909356989, 'lr': 0.00266022648583618, 'weight_decay': 3.228977758326509e-05}
Best AUC: 0.920156341320865


In [15]:

import optuna.visualization as vis

fig = vis.plot_optimization_history(study)
fig.show()
